In [ ]:
pip install transformers datasets sentence-transformers wikipedia torch scikit-learn tqdm matplotlib pandas


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
 

In [ ]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import wikipedia
import pandas as pd
import os
import json
from tqdm import tqdm

os.environ["WANDB_DISABLED"] = "true"

# Load TruthfulQA dataset
dataset = load_dataset("truthful_qa", "generation")
questions = [item["question"] for item in dataset["validation"]]

# Generator model
generator = pipeline("text-generation", model="gpt2", max_new_tokens=60)

# Caching paths
gen_cache_path = "generated_answers.json"
evidence_cache_path = "wiki_evidence.json"

# Load cache if exists
generated_data = {}
if os.path.exists(gen_cache_path):
    with open(gen_cache_path) as f:
        generated_data = json.load(f)
else:
    for question in tqdm(questions, desc="Generating answers"):
        try:
            answer = generator(question)[0]["generated_text"]
            generated_data[question] = answer
        except:
            generated_data[question] = "Error generating answer"
    with open(gen_cache_path, "w") as f:
        json.dump(generated_data, f)

# Wikipedia evidence retrieval with caching
evidence_data = {}
if os.path.exists(evidence_cache_path):
    with open(evidence_cache_path) as f:
        evidence_data = json.load(f)
else:
    for question in tqdm(questions, desc="Retrieving Wikipedia evidence"):
        try:
            summary = wikipedia.summary(question, sentences=2)
        except:
            summary = "No relevant Wikipedia summary found."
        evidence_data[question] = summary
    with open(evidence_cache_path, "w") as f:
        json.dump(evidence_data, f)

# Sentence-BERT similarity model
sim_model = SentenceTransformer("all-MiniLM-L6-v2")

def is_consistent_with_evidence(answer, evidence, threshold=0.7):
    embedding1 = sim_model.encode(answer, convert_to_tensor=True)
    embedding2 = sim_model.encode(evidence, convert_to_tensor=True)
    score = util.cos_sim(embedding1, embedding2).item()
    return 1 if score >= threshold else 0

# Build labeled dataset
labeled_data = []
for q in tqdm(questions, desc="Labeling"):
    ans = generated_data[q]
    ev = evidence_data[q]
    label = is_consistent_with_evidence(ans, ev)
    labeled_data.append({
        "question": q,
        "answer": ans,
        "evidence": ev,
        "label": label
    })

# Convert to DataFrame
df = pd.DataFrame(labeled_data)

# Tokenization
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["question"],
        examples["answer"] + " [SEP] " + examples["evidence"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )

tokenized = df.apply(tokenize_function, axis=1)
input_ids = list(tokenized.apply(lambda x: x["input_ids"]))
attention_mask = list(tokenized.apply(lambda x: x["attention_mask"]))
labels = df["label"].tolist()

# Dataset
class HallucinationDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "attention_mask": torch.tensor(self.attention_mask[idx]),
            "labels": torch.tensor(self.labels[idx])
        }

train_ids, val_ids, train_mask, val_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42)

train_dataset = HallucinationDataset(train_ids, train_mask, train_labels)
val_dataset = HallucinationDataset(val_ids, val_mask, val_labels)

# Model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=1)
    return {"accuracy": accuracy}

# TrainingArguments
training_args = TrainingArguments(
    output_dir="./hallucination_detector_full",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
eval_results = trainer.evaluate()
print(eval_results)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


{'eval_loss': 0.10383295267820358, 'eval_accuracy': 0.9817073170731707, 'eval_precision': 1.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_runtime': 291.9748, 'eval_samples_per_second': 0.562, 'eval_steps_per_second': 0.072, 'epoch': 3.0}


In [ ]:
training_args = TrainingArguments(
    output_dir="./hallucination_detector_full",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
eval_results = trainer.evaluate()
print(eval_results)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


{'eval_loss': 0.10587287694215775, 'eval_accuracy': 0.9817073170731707, 'eval_precision': 1.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_runtime': 4.5614, 'eval_samples_per_second': 35.954, 'eval_steps_per_second': 4.604, 'epoch': 3.0}
